In [ ]:
! pip install selenium

In [ ]:
! pip install beautifulsoup4

In [ ]:
!pip install openpyxl

In [ ]:
pip install pandas

# 인스타그램 크롤링 과정

In [ ]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import re, unicodedata


driver = webdriver.Chrome('./chromedriver.exe')
url = 'https://www.instagram.com'
driver.get(url)
time.sleep(5)

email = 'openss_1234'
input_id = driver.find_elements(By.CSS_SELECTOR, 'input')[0]
input_id.clear()
input_id.send_keys(email)


password = 'chlxodud0720!'
input_pw = driver.find_elements(By.CSS_SELECTOR, 'input')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

def insta_search(word):           # 인스타그램 특정 해시태그 검색
    url = 'https://www.instagram.com/explore/tags/' + word
    return url 

url = insta_search('대구맛집')
driver.get(url)
time.sleep(5)




def select_first(driver):          # 첫번째 게시물 선택 
    time.sleep(5)
    first=driver.find_elements(By.TAG_NAME,'div')[1]
    time.sleep(5)
    first.click()
    time.sleep(5)

select_first(driver)



def get_content(driver):          # 본문,태그,날짜,좋아요 수집
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try:
        content = soup.select('div._a9zs > h1')[0].text  
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    
    tags = re.findall(r'#[^s#,\\]+',content)

    date = soup.select('time._aaqe')[0]['datetime'][:10]
    try:
        like = soup.select('div >span > a > span')[0].text
    except:
        like = 0
    
    try:
        place = soup.select('div._aacl._aacn._aacu._aacy._aada._aade')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    
    data = [content, date, like, place, tags]
    return data 


def move_next(driver):            # 다음 게시글 조회
    time.sleep(2)
    right = driver.find_element(By.TAG_NAME,'html')    
    right.send_keys(Keys.ARROW_RIGHT)
    time.sleep(2)


results = [ ] 

target = 5
for i in range(target):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)   
    except:
        time.sleep(2)
        move_next(driver)
    


import pandas as pd               # csv 파일로 저장

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'place', 'tags']
results_df.to_excel('./insta_hotp_crawling.xlsx',index=False)


